In [1]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("PreProcessing")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/08/07 10:53:04 WARN Utils: Your hostname, Bobert resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/07 10:53:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/07 10:53:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# initializing data
ytaxi = spark.read.parquet('../data/raw/YellowTaxi/yellow_tripdata_2023-12.parquet','../data/raw/YellowTaxi/yellow_tripdata_2024-01.parquet',
                           '../data/raw/YellowTaxi/yellow_tripdata_2024-02.parquet','../data/raw/YellowTaxi/yellow_tripdata_2024-03.parquet',
                           '../data/raw/YellowTaxi/yellow_tripdata_2024-04.parquet','../data/raw/YellowTaxi/yellow_tripdata_2024-05.parquet')

In [5]:
# Converting Data Types
ytaxi.printSchema()

ytaxi.groupBy('passenger_count').mean('tip_amount')

columns_to_check=['VendorID', 'passenger_count','payment_type']
for column in columns_to_check:
    print(set(ytaxi.select(column).rdd.flatMap(lambda x: x).collect()))

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



{1, 2, 6}


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, None}


{0, 1, 2, 3, 4, 5}


In [5]:
# changing data types to be slightly more efficient
# passenger_count could be int, store_and_flag bool, payment type int, can turn airport fee into bool since it is
# either 1.75 or 0
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType


ytaxi2 = ytaxi

ytaxi2 = ytaxi2.withColumn('passenger_count',col('passenger_count').cast(IntegerType()))
ytaxi2 = ytaxi2.replace(['Y','N'],['True','False'],subset='store_and_fwd_flag')
ytaxi2 = ytaxi2.withColumn('store_and_fwd_flag',col('store_and_fwd_flag').cast(BooleanType()))
ytaxi2 = ytaxi2.withColumn('payment_type',col('payment_type').cast(IntegerType()))
ytaxi2 = ytaxi2.withColumn('airport_fee',col('airport_fee').cast(BooleanType()))

#airport_fee is changed to bool even though there were negative values, they are all -1.75 which I assumed to be 1.75

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: boolean (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: boolean (nullable = true)



VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2023-12-01 00:06:06,2023-12-01 00:15:47,0,1.1,1,false,230,48,1,10.0,3.5,0.5,1.5,0.0,1.0,16.5,2.5,false
1,2023-12-01 00:22:26,2023-12-01 00:28:53,0,1.5,1,false,142,238,1,9.3,3.5,0.5,2.85,0.0,1.0,17.15,2.5,false
1,2023-12-01 00:59:44,2023-12-01 01:13:22,2,2.2,1,false,114,186,1,13.5,3.5,0.5,3.0,0.0,1.0,21.5,2.5,false
2,2023-12-01 00:22:17,2023-12-01 00:30:59,1,0.66,1,false,79,79,2,7.2,1.0,0.5,0.0,0.0,1.0,12.2,2.5,false
2,2023-12-01 00:18:16,2023-12-01 00:25:32,2,2.2,1,false,229,263,1,11.4,1.0,0.5,2.0,0.0,1.0,18.4,2.5,false
1,2023-12-01 00:13:17,2023-12-01 00:23:53,0,5.7,1,false,88,141,1,23.3,3.5,0.5,0.0,0.0,1.0,28.3,2.5,false
2,2023-12-01 00:17:09,2023-12-01 00:33:31,1,5.33,1,false,45,162,1,24.7,1.0,0.5,3.0,0.0,1.0,32.7,2.5,false
2,2023-12-01 00:40:49,2023-12-01 00:44:10,1,0.76,1,false,170,107,1,5.8,1.0,0.5,1.0,0.0,1.0,11.8,2.5,false
2,2023-12-01 00:19:04,2023-12-01 00:34:36,1,3.33,1,false,186,209,1,17.7,1.0,0.5,3.4,0.0,1.0,26.1,2.5,false
1,2023-12-01 00:08:39,2023-12-01 00:16:18,1,2.1,1,false,163,262,1,12.1,3.5,0.5,3.42,0.0,1.0,20.52,2.5,false


In [8]:
ytaxi2.write.parquet('../data/curated/YellowTaxi_Checkpt1')